In [1]:
import wave
import json
import numpy as np
import cv2

from vosk import Model, KaldiRecognizer, SetLogLevel
import Word as custom_Word

import pandas as pd

In [2]:
def wordToLetter(word):
    letter_len = (word.end-word.start)/len(word.word)
    letter_list = []
    i = 0
    while i < len(word.word):
        if word.word[i] == 'a' or word.word[i] == 'e' or word.word[i] == 'i':
            letter = ['a', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'o':
            letter = ['o', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'd' or word.word[i] == 'n' or word.word[i] == 'x' or word.word[i] == 'y' or word.word[i] == 'z':
            letter = ['c', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'g' or word.word[i] == 'k':
            letter = ['g', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'l':
            letter = ['l', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'b' or word.word[i] == 'm' or word.word[i] == 'p':
            letter = ['b', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'f' or word.word[i] == 'v':
            letter = ['f', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'j':
            letter = ['j', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'u':
            letter = ['u', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'q' or word.word[i] == 'w':
            letter = ['q', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
            letter_list.append(letter)
        elif word.word[i] == 'c':
            if i < (len(word.word)-1) and word.word[i+1] == 'h':
                letter = ['j', word.start+(i*letter_len), word.start+((i+2)*letter_len)]
                letter_list.append(letter)
                i += 1
            else :
                letter = ['c', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
                letter_list.append(letter)
        elif word.word[i] == 's':
            if i < (len(word.word)-1) and word.word[i+1] == 'h':
                letter = ['j', word.start+(i*letter_len), word.start+((i+2)*letter_len)]
                letter_list.append(letter)
                i += 1
            else :
                letter = ['c', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
                letter_list.append(letter)
        elif word.word[i] == 't':
            if i < (len(word.word)-1) and word.word[i+1] == 'h':
                letter = ['t', word.start+(i*letter_len), word.start+((i+2)*letter_len)]
                letter_list.append(letter)
                i += 1
            else :
                letter = ['c', word.start+(i*letter_len), word.start+((i+1)*letter_len)]
                letter_list.append(letter)
        i += 1
    return letter_list

In [3]:
def audioToLetters(model_path, audio_filename):
    model = Model(model_path)
    wf = wave.open(audio_filename, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    # get the list of JSON dictionaries
    results = []
    # recognize speech using vosk model
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part_result = json.loads(rec.Result())
            results.append(part_result)
    part_result = json.loads(rec.FinalResult())
    results.append(part_result)

    # convert list of JSON dictionaries to list of 'Word' objects
    list_of_Words = []
    for sentence in results:
        if len(sentence) == 1:
            # sometimes there are bugs in recognition 
            # and it returns an empty dictionary
            # {'text': ''}
            continue
        for obj in sentence['result']:
            w = custom_Word.Word(obj)  # create custom Word object
            list_of_Words.append(w)  # and add it to list

    wf.close()  # close audiofile

    result_list = []
    for elt in list_of_Words:
        letter_list = wordToLetter(elt)
        for letter in letter_list:
            result_list.append(letter)
    return result_list

In [4]:
model_path = "models/vosk-model-en-us-0.21"
audio_filename = "C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/fadg0/audio/sx379.wav"

letter_list = audioToLetters(model_path, audio_filename)
index = list(range(len(letter_list)))
df = pd.DataFrame(0, index=index,columns=['a', 'o', 'c', 'g', 'l', 'b', 'f', 't', 'j', 'u', 'q', 'start', 'end'])

i = 0
for elt in letter_list:
    df.loc[i, elt[0]] = 1
    df.loc[i, 'start'] = int(elt[1]*24)
    df.loc[i, 'end'] = int(elt[2]*24)
    i += 1

df.to_csv("C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/fadg0/phoneme_list/sx379.csv", index=False)

In [5]:
files = ['sa1', 'sa2', 'si1279', 'si1909', 'si649', 'sx109', 'sx19', 'sx199', 'sx289', 'sx379']

In [105]:
data = pd.DataFrame(0, index=index,columns=['speaker', 'record', 'phoneme', 'face_mesh_path'])
y = 0
for file in files :
    df = pd.read_csv("C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/fadg0/phoneme_list/"+ file + ".csv")
    for i in range(len(df)):
        phoneme_index = np.where(df.iloc[i, :] == 1)[0][0]
        frame = int((df.loc[i, 'start'] + df.loc[i, 'end'])/2)
        frame = '{:03}'.format(frame)
        fmpath = "fadg0/face_mesh/" + file + "/face_" + frame 
        data.loc[y]=['fadg0', file, float(phoneme_index), fmpath]
        y += 1

In [18]:
frame = 2
frame = '{:03}'.format(frame)
print(frame)

002


In [106]:
data.head()

,speaker,record,phoneme,face_mesh_path
0,fadg0,sa1,8.0,fadg0/face_mesh/sa1/face_018
1,fadg0,sa1,0.0,fadg0/face_mesh/sa1/face_021
2,fadg0,sa1,0.0,fadg0/face_mesh/sa1/face_025
3,fadg0,sa1,2.0,fadg0/face_mesh/sa1/face_027
4,fadg0,sa1,2.0,fadg0/face_mesh/sa1/face_028


In [9]:
S = 0
for file in files :
    df = pd.read_csv("C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/fadg0/phoneme_list/"+ file + ".csv")
    print(df.head())
    print(len(df))
    S += len(df)
    print(S)

   a  o  c  g  l  b  f  t  j  u  q  start  end
0  0  0  0  0  0  0  0  0  1  0  0     16   20
1  1  0  0  0  0  0  0  0  0  0  0     20   23
2  1  0  0  0  0  0  0  0  0  0  0     25   26
3  0  0  1  0  0  0  0  0  0  0  0     26   28
4  0  0  1  0  0  0  0  0  0  0  0     28   29
39
39
   a  o  c  g  l  b  f  t  j  u  q  start  end
0  0  0  1  0  0  0  0  0  0  0  0     13   15
1  0  1  0  0  0  0  0  0  0  0  0     15   16
2  0  0  1  0  0  0  0  0  0  0  0     16   18
3  0  0  1  0  0  0  0  0  0  0  0     20   21
4  1  0  0  0  0  0  0  0  0  0  0     21   24
29
68
   a  o  c  g  l  b  f  t  j  u  q  start  end
0  0  0  0  1  0  0  0  0  0  0  0     14   16
1  1  0  0  0  0  0  0  0  0  0  0     18   21
2  1  0  0  0  0  0  0  0  0  0  0     21   23
3  0  0  0  1  0  0  0  0  0  0  0     23   25
4  0  0  1  0  0  0  0  0  0  0  0     25   28
19
87
   a  o  c  g  l  b  f  t  j  u  q  start  end
0  0  0  0  0  0  0  0  1  0  0  0     25   28
1  1  0  0  0  0  0  0  0  0  0  0     28 

In [107]:
data.to_csv("data.csv", index=False)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [12]:
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

In [13]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [14]:
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [15]:
print(clf.predict([[0, 0, 0, 0]]))

[1]


In [25]:
import csv 

mesh_path = data.loc[0, 'face_mesh_path']
path = "C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/" + data.loc[0, 'face_mesh_path']
f = open(path, 'r')
# lire le contenu du fichier
r = csv.reader(f, delimiter=" ", quoting=csv.QUOTE_NONNUMERIC)
liste = list(r)
f.close()
print(len(liste))

478


In [94]:
X = []
y = []
for i in range(350):
    path = "C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/" + data.loc[i, 'face_mesh_path']
    f = open(path, 'r')
    # lire le contenu du fichier
    r = csv.reader(f, delimiter=" ", quoting=csv.QUOTE_NONNUMERIC)
    face_mesh = list(r)
    f.close()
    X.append(face_mesh[0])
    y.append(data.loc[i, 'phoneme'])

In [95]:
model = RandomForestClassifier()

In [96]:
model.fit(X, y)

RandomForestClassifier()

In [97]:
for i in range(350, 357):
    path = "C:/Users/Enzo.Magal/Documents/Enzo2021/VidTIMIT/" + data.loc[i, 'face_mesh_path']
    f = open(path, 'r')
    # lire le contenu du fichier
    r = csv.reader(f, delimiter=" ", quoting=csv.QUOTE_NONNUMERIC)
    face_mesh = list(r)
    f.close()
    print(model.predict([face_mesh[0]]))

['a']
['c']
['a']
['o']
['c']
['a']
['a']


In [98]:
[face_mesh[0]]

[[0.4986247718334198, 0.5527295470237732, -0.03650111332535744]]

In [99]:
print(np.shape(X))
print(X[0][0])
print(np.shape(X[0]))

(350, 3)
0.5020045638084412
(3,)


In [100]:
data.tail(10)

,speaker,record,phoneme,face_mesh_path
347,fadg0,sx379,c,fadg0/face_mesh/sx379/face_055
348,fadg0,sx379,o,fadg0/face_mesh/sx379/face_057
349,fadg0,sx379,c,fadg0/face_mesh/sx379/face_060
350,fadg0,sx379,t,fadg0/face_mesh/sx379/face_064
351,fadg0,sx379,u,fadg0/face_mesh/sx379/face_068
352,fadg0,sx379,c,fadg0/face_mesh/sx379/face_073
353,fadg0,sx379,c,fadg0/face_mesh/sx379/face_076
354,fadg0,sx379,a,fadg0/face_mesh/sx379/face_079
355,fadg0,sx379,c,fadg0/face_mesh/sx379/face_081
356,fadg0,sx379,c,fadg0/face_mesh/sx379/face_084
